# Utility functions used throughout analysis

In [ ]:
Import necessary libraries
import collections
from datetime import datetime, UTC
from inspect import getsourcefile
import itertools
import litstudy  # Use pip install git+https://github.com/NLeSC/litstudy to download dev version
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pickle
from pyzotero import zotero
import re
import requests
import shutil
from typing import List, Union, Dict, Any, Optional
from urllib.parse import urlparse

In [4]:
def regex_list_files(in_dir: Union[str, Path], in_pattern: str, full_path: bool = True) -> List[str]:
    """
    Lists files in a directory matching a regular expression.

    Args:
        in_dir: The directory to search. Can be a string path or a pathlib.Path object.
        in_pattern: The regular expression pattern to match filenames against.
        full_path: Whether to return full paths (True) or just filenames (False).

    Returns:
        A list of strings, either full paths or filenames, of files matching the pattern.
        Returns an empty list if no matches are found or if an error occurs.

    Raises:
        TypeError: if input arguments are of incorrect type
        ValueError: if input directory does not exist
        re.error:  If the regular expression pattern is invalid.

    """

    # --- Input Type Validation ---
    if not isinstance(in_dir, (str, Path)):
        raise TypeError("in_dir must be a string or pathlib.Path object.")
    if not isinstance(in_pattern, str):
        raise TypeError("in_pattern must be a string.")
    if not isinstance(full_path, bool):
        raise TypeError("full_path must be a boolean.")

    # --- Convert to Path object for consistency ---
    if isinstance(in_dir, str):
        in_dir = Path(in_dir)

    # --- Input Value Validation ---
    if not in_dir.is_dir():
        raise ValueError(f"The directory '{in_dir}' does not exist.")

    # --- Regex Compilation (with error handling) ---
    try:
        regex = re.compile(in_pattern)
    except re.error as e:
        raise re.error(f"Invalid regular expression pattern: {e}")

    file_list = []
    for root, _, files in os.walk(in_dir):  # os.walk works with Path objects
        for file in files:
            if regex.match(file):
                if full_path:
                    # Use .joinpath for consistent path construction with Path objects
                    file_list.append(str(Path(root).joinpath(file)))  # Convert to string for consistent return type
                else:
                    file_list.append(file)

    return file_list

In [6]:
# Read and compile references from the WoS search into a single document set (lit_study format)
def rpickle_bibdocset(in_dirpath, in_pattern, out_pickle):
    if not out_pickle.exists():
        # Get list of every bib file
        bib_initlist = [p for p in list(in_dirpath.glob('*')) 
                        if re.compile(in_pattern).match(str(p))]
        # Read bib files from first scoping and join them (takes ~15-20 sec/1000 refs)
        reflist = []
        for bib in bib_initlist:
            reflist += litstudy.load_bibtex(bib)

        # Pickle them (save the full document set as a binary file on disk that can be easily retrieved)
        with open(out_pickle, 'wb') as f:
            pickle.dump(reflist, f)
    else:
        # Read pre-saved document set
        with open(out_pickle, 'rb') as f:
            reflist = pickle.load(f)
    return reflist

# Get titles and DOIs from Zotero test list
def get_testlist(library_id, api_key_path, test_list_name):
    api_key = api_key_path.read_text().strip()
    zot = zotero.Zotero(library_id=library_id, 
                        library_type='group', 
                        api_key=api_key)
    testlist_colID = str([col['key'] for col in zot.collections_top() 
                          if col['data']['name'] == test_list_name][0])
    testlist_items = zot.everything(zot.collection_items_top(testlist_colID))

    testlist_title_dois = collections.defaultdict(list)
    for ref in testlist_items:
        testlist_title_dois[ref['key']].append(ref['data']['title'])
        if 'DOI' in ref['data']:
            testlist_title_dois[ref['key']].append(ref['data']['DOI'])
        else:
            testlist_title_dois[ref['key']].append(np.nan)
    return testlist_title_dois

# Get all DOIs and titles in references returned from search
def tabulate_searchlist(in_reflist, out_csvpath):
    if not out_csvpath.exists():
        reflist_dict = {}
        for i, ref in enumerate(in_reflist):
            reflist_dict[i] = [re.sub(r"[^a-zA-Z\d\s]", "", 
                                      ref.title.replace('\n', ' ').lower()),
                               ref.publication_source,
                               ref.publication_year, ref.abstract]
            if 'doi' in ref.entry:
                reflist_dict[i].append(ref.entry['doi'])
            else:
                reflist_dict[i].append(np.nan)

        reflist_pd = pd.DataFrame.from_dict(reflist_dict, orient='index')
        reflist_pd.columns = ['title', 'source', 'year', 'abstract', 'doi']
        reflist_pd.to_csv(out_csvpath)
    else:
        reflist_pd = pd.read_csv(out_csvpath)
    return reflist_pd

# Erite string y to file x
def write(x, y):
    with open(x, 'a') as f:
        f.write(y)
        f.write('\n')
    return _

def combine_2w_regex(pattern1, pattern2, precede=False):
    """
    precede = False means word1 and word2 are looked at with either being first word
    precede = True means word1 must be first, word2 must be second
    """
    regexp = f"{pattern1}\\W{pattern2}\\b"
    if precede == False:
        regexp = f"({regexp})|({pattern2}\\W{pattern1}\\b)"
    return regexp

# Count number a times a simple 2-pattern group occurs in text
def find_2w_regex(text, pattern1, pattern2, precede=False):
    """
    precede = False means word1 and word2 are looked at with either being first word
    precede = True means word1 must be first, word2 must be second
    """
    regexp = f"{pattern1}\\W{pattern2}\\b"
    if precede == False:
        regexp = f"({regexp})|({pattern2}\\W{pattern1}\\b)"
    wa = re.findall(regexp, text)
    wal = len(wa)
    return wal

# Count number a times a simple pattern occurs in text
def find_regex(text, pattern):
    return len(re.findall(f"{pattern}", text))

# Join all strings in a list with | signs and parentheses
def recomb(in_str, recomb_sep):
    if isinstance(in_str, list):
        return f"({recomb_sep.join(f'({w})' for w in in_str)})"
    else:
        return in_str

# Find patterns in text based on search dictionary
def combo_refind(in_searchdict, text):
    for regexp_combo in in_searchdict.values():
        if regexp_combo[0] == 'with':
            k = find_2w_regex(text, 
                              regexp_combo[1][0], 
                              regexp_combo[1][1], 
                              precede=False)
        elif regexp_combo[0] == 'pre':
            k = find_2w_regex(text,
                              regexp_combo[1][0], 
                              regexp_combo[1][1], 
                              precede=True)
        elif regexp_combo[0] is None:
            k = find_regex(text, regexp_combo)
        else:
            break
        return k

# Generate n-grams from DOI
#CHECK OUT: from pattern.en import ngrams
#print(ngrams("He goes to hospital", n=2))
def DOI_ngram(A):
    count0 = collections.Counter()
    s1 = A[0].replace("'", '')
    s2 = s1.replace("?", '')
    s3 = s2.replace(".", '')
    s4 = s3.replace(",", '')
    s5 = s4.replace(":", '')
    s6 = s5.lower()
    tokens = nltk.word_tokenize(s6)
    every = nltk.everygrams(tokens, 2, 4)
    count0 = count0 + (collections.Counter(every))
    count0 = count0.most_common()

    count1 = collections.Counter()
    s1 = A[1].replace("'", '')
    s2 = s1.replace("?", '')
    s3 = s2.replace(".", '')
    s4 = s3.replace(",", '')
    s5 = s4.replace(":", '')
    s6 = s5.lower()
    tokens = nltk.word_tokenize(s6)
    every = nltk.everygrams(tokens, 2, 4)
    count1 = count1 + (collections.Counter(every))
    count1 = count1.most_common()

    count2 = collections.Counter()
    for idx, i in enumerate(A[2]):
        x = collections.Counter([l.lower() for l in i])
        count2 += x
    count2 = count2.most_common()

    count3 = count0 + count1 + count2
    return count3

In [8]:
def list_of_dicts_to_dataframe(
    list_of_dicts: List[Dict[str, Any]], 
    keys_to_keep: Optional[List[str]] = None
) -> pd.DataFrame:
    """
    Converts a list of dictionaries to a Pandas DataFrame, keeping a specified
    subset of keys as columns.  Handles missing keys gracefully.

    Args:
        list_of_dicts: The list of dictionaries.
        keys_to_keep: An optional list of keys to keep as columns.
            If None, attempts to use all keys present in *any* of the
            dictionaries, but prioritizes keys from the *first* dictionary
            if there are inconsistencies.

    Returns:
        A Pandas DataFrame.

    Raises:
        TypeError: if input is not a list or contains non-dict elements.
        ValueError: if keys_to_keep is provided but is empty.
    """

    if not isinstance(list_of_dicts, list):
        raise TypeError("Input 'list_of_dicts' must be a list of dictionaries.")
    if not all(isinstance(item, dict) for item in list_of_dicts):
        raise TypeError("All elements in 'list_of_dicts' must be dictionaries.")
    if keys_to_keep is not None and not isinstance(keys_to_keep, list):
        raise TypeError("'keys_to_keep' must be a list of strings or None.")
    if keys_to_keep is not None and len(keys_to_keep) == 0:
        raise ValueError("'keys_to_keep' cannot be an empty list.")


    if keys_to_keep is None:
        # Attempt to use all keys, prioritizing the first dictionary
        if not list_of_dicts:  # Handle empty input list
            return pd.DataFrame()
        keys_to_keep = list(list_of_dicts[0].keys())  # Start with keys from first dict
        # Add any keys present in *other* dicts but missing from the first.
        for item in list_of_dicts:
            for key in item:
                if key not in keys_to_keep:
                    keys_to_keep.append(key)
    
    # Create the DataFrame, handling missing keys
    df_data = []
    for item in list_of_dicts:
        row = {key: item.get(key, None) for key in keys_to_keep}
        df_data.append(row)

    df = pd.DataFrame(df_data, columns=keys_to_keep)
    return df

In [10]:
#Function to export OpenAlex works to RIS
%run lib/openalex_formatter_ris.py

def export_oalex_works_to_ris(
    works: List[Dict[str, Any]], 
    filename: str) -> None:
    """
    Export a list of OpenAlex works to an RIS file, including the abstract.

    Args:
        works: List of OpenAlex work dictionaries.
        filename: The name of the RIS file to be created.
    """

    if not isinstance(works, list):
        raise TypeError("works must be a list of dictionaries.")
    if not isinstance(filename, str):
        raise TypeError("filename must be a string.")
    if not filename.endswith(".ris"):
        logging.warning("Filename does not end with '.ris'.  This may cause problems with some RIS readers.")

    try:
        with open(filename, 'w', encoding='utf-8') as ris_file:
            for work in works:
                if not isinstance(work, dict):
                    print(f"Skipping invalid work entry (not a dictionary): {work}")
                    continue

                ris_entry = build_ris_entry(work)
                ris_file.write(ris_entry)

                # # --- Article type ----
                # ris_file.write("TY  - JOUR\n")  # Type of reference (Journal Article)
                
                # # --- Title ---
                # title = work.get('title', '') or ''  # Handle None or missing title
                # ris_file.write(f"TI  - {title}\n")

                # # --- Authors ---
                # # Handle authors correctly.  OpenAlex stores authors as a list of dictionaries.
                # authors = work.get('authorships', []) or []
                # for author_data in authors:
                #     author_name = author_data.get('author', {}).get('display_name', '') or ''
                #     if author_name:
                #         ris_file.write(f"AU  - {author_name}\n")

                # # --- Publication Year ---
                # year = work.get('publication_year', '') or ''
                # ris_file.write(f"PY  - {year}\n")

                # # --- Journal (Source) ---
                # #  OpenAlex stores source information in 'primary_location' and 'locations'.
                # source_title = ''
                # if work.get('primary_location') and work.get('primary_location').get('source'):
                #     source_title = work['primary_location']['source'].get('display_name', '') or ''
                # elif work.get('locations'):
                #     for location in work['locations']:
                #         if location.get('source'):
                #             source_title = location['source'].get('display_name', '') or ''
                #             break # Use the first available location.

                # ris_file.write(f"JO  - {source_title}\n")

                # # --- Volume, Issue, Pages ---
                # volume = work.get('volume', '') or ''
                # issue = work.get('issue', '') or ''
                # #  Pages can be in 'biblio' or as separate 'page_start', 'page_end'
                # start_page = work.get('page_start', '') or ''
                # end_page = work.get('page_end', '') or ''
                # if not start_page and work.get('biblio'):
                #     start_page = work['biblio'].get('first_page', '') or ''
                #     end_page = work['biblio'].get('last_page', '') or ''
                
                # ris_file.write(f"VL  - {volume}\n")
                # ris_file.write(f"IS  - {issue}\n")
                # ris_file.write(f"SP  - {start_page}\n")
                # ris_file.write(f"EP  - {end_page}\n")

                # # --- DOI ---
                # doi = work.get('doi', '') or ''  # Get DOI directly.  It's a top-level field.
                # ris_file.write(f"DO  - {doi}\n")

                # # --- Abstract ---
                # abstract = work['abstract']
                # if abstract:  # Only write abstract if it exists
                #     ris_file.write(f"AB  - {abstract}\n")


                # ris_file.write("ER  - \n\n")  # End of reference

    except (IOError, OSError) as e:
        raise OSError(f"Error writing to RIS file: {e}")
    except Exception as e: #Catch remaining exceptions
        raise Exception(f"An unexpected error occurred: {e}")

In [12]:
def searching(query_string, broad='no', has_ngrams='true', 
              does_date_matter='no', from_date='1976-01-01'):
    if broad == 'no':
        if does_date_matter == 'yes':
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},\
                has_ngrams:{has_ngrams},from_publication_date:{from_date},to_publication_date:{to_date}&page=1&per-page=20'
            ).json()
        else:
            institution = requests.get(
                f'https://api.openalex.org/works?filter=title.search:{query_string},\
                has_ngrams:{has_ngrams},&page=1&per-page=20'
            ).json()
    elif broad == 'yes':
        institution = requests.get(
            f'https://api.openalex.org/works?search={query_string}&page=1&per-page=20'
        ).json()
    return institution